In [6]:

        
import json
from ConfigParser import SafeConfigParser
from operator import add
import matplotlib.pyplot as plt 
from random import randint

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from util import save_rdd, get_parser

#BUG hadoop fs -put contributions.txt /user/ellery/en-es/contributions.txt
config = '/home/ellery/wikimedia/missing_articles/missing_articles.ini'
cp = SafeConfigParser()
cp.read(config)

exp_dir = 'en-es'
MIN_BYTES = 100
base_dir = os.path.join(cp.get('general', 'local_data_dir'), exp_dir)
hadoop_base_dir = os.path.join(cp.get('general', 'hadoop_data_dir'), exp_dir)
contribution_file = os.path.join(hadoop_base_dir, cp.get('eval', 'contributions')) 



In [7]:
contribution_file

'/user/ellery/en-es/contributions.txt'

In [8]:
def reduce_contributions(line):
    l = json.loads(line)
    l['contributions'] = [x for x in l['contributions'] if float(x['bytes_added']) >= MIN_BYTES]
    return l


contributions = sc.textFile(contribution_file)\
.map(reduce_contributions)\
.filter(lambda x: len(x['contributions']) >=2)\
.map(lambda x: (x, randint(0,1)))

train = contributions.filter(lambda x: x[1]==0).map(lambda x: json.dumps(x[0]))
save_rdd(train, base_dir , hadoop_base_dir, cp.get('eval', 'train'))



256
0
256
256
0
0
0
0


In [9]:
test = contributions.filter(lambda x: x[1]==1).map(lambda x: json.dumps(x[0]))
save_rdd(test, base_dir , hadoop_base_dir, cp.get('eval', 'test'))

256
256
256
256
0
0
0
0
